# Python Basics Part 2 - Advanced Concepts

Deeper Python concepts for scientific computing

## Advanced Function Concepts

In [ ]:
# *args and **kwargs
def analyze_data(*measurements, **metadata):
    """Function that accepts variable arguments"""
    print(f"Processing {len(measurements)} measurements")
    print(f"Metadata: {metadata}")
    
    if measurements:
        avg = sum(measurements) / len(measurements)
        return {
            'count': len(measurements),
            'average': avg,
            'metadata': metadata
        }
    return None

# Usage examples
result1 = analyze_data(23.1, 24.5, 22.8, instrument='spectrometer', temp=25)
result2 = analyze_data(1.2, 1.5, 1.1, 1.8, 1.3, experiment='pH_test', date='2023-01-01')

print("Result 1:", result1)
print("Result 2:", result2)

# Function annotations (type hints)
from typing import List, Dict, Optional, Union

def calculate_concentration(absorbance: float, 
                         extinction_coeff: float, 
                         path_length: float = 1.0) -> float:
    """Calculate concentration using Beer's Law.
    
    Args:
        absorbance: Measured absorbance
        extinction_coeff: Molar extinction coefficient
        path_length: Path length in cm (default 1.0)
    
    Returns:
        Concentration in mol/L
    """
    return absorbance / (extinction_coeff * path_length)

concentration = calculate_concentration(0.5, 1000, 1.0)
print(f"Concentration: {concentration} mol/L")

# Closure example
def create_calibration_curve(slope: float, intercept: float):
    """Returns a function that applies calibration"""
    def calibrate(raw_value: float) -> float:
        return slope * raw_value + intercept
    return calibrate

# Create specific calibration functions
temp_calibrator = create_calibration_curve(0.98, 2.1)
pressure_calibrator = create_calibration_curve(1.05, -0.3)

print(f"Calibrated temperature: {temp_calibrator(25.0)}")
print(f"Calibrated pressure: {pressure_calibrator(100.0)}")

## Decorators

In [ ]:
import time
import functools
from typing import Callable, Any

# Timing decorator
def timing_decorator(func: Callable) -> Callable:
    """Decorator to measure function execution time"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

# Validation decorator
def validate_positive(func: Callable) -> Callable:
    """Decorator to validate that arguments are positive"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        for arg in args:
            if isinstance(arg, (int, float)) and arg <= 0:
                raise ValueError(f"Argument {arg} must be positive")
        return func(*args, **kwargs)
    return wrapper

# Memoization decorator
def memoize(func: Callable) -> Callable:
    """Cache function results"""
    cache = {}
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        key = str(args) + str(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    
    wrapper.cache = cache  # Expose cache for inspection
    return wrapper

# Using decorators
@timing_decorator
@validate_positive
def complex_calculation(n: int) -> float:
    """Simulate a complex calculation"""
    total = 0
    for i in range(n):
        total += i ** 0.5
    return total

@memoize
def factorial(n: int) -> int:
    """Calculate factorial with memoization"""
    if n <= 1:
        return 1
    return n * factorial(n - 1)

# Test decorators
result = complex_calculation(100000)
print(f"Complex calculation result: {result:.2f}")

# Test memoization
print(f"Factorial 5: {factorial(5)}")
print(f"Factorial 10: {factorial(10)}")
print(f"Cache size: {len(factorial.cache)}")

# Try with invalid input
try:
    complex_calculation(-5)
except ValueError as e:
    print(f"Validation error: {e}")

## Context Managers

In [ ]:
from contextlib import contextmanager
import os
import tempfile

# Custom context manager class
class ExperimentTimer:
    def __init__(self, experiment_name: str):
        self.experiment_name = experiment_name
        self.start_time = None
        
    def __enter__(self):
        print(f"Starting experiment: {self.experiment_name}")
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        duration = time.time() - self.start_time
        print(f"Experiment '{self.experiment_name}' completed in {duration:.2f} seconds")
        if exc_type:
            print(f"Exception occurred: {exc_val}")
        return False  # Don't suppress exceptions

# Using the context manager
with ExperimentTimer("pH Measurement"):
    # Simulate experiment
    time.sleep(0.1)
    measurements = [7.2, 7.1, 7.3, 7.0]
    avg_ph = sum(measurements) / len(measurements)
    print(f"Average pH: {avg_ph}")

# Context manager using generator function
@contextmanager
def temporary_directory():
    """Create and cleanup temporary directory"""
    temp_dir = tempfile.mkdtemp()
    print(f"Created temporary directory: {temp_dir}")
    try:
        yield temp_dir
    finally:
        # Cleanup
        import shutil
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temporary directory: {temp_dir}")

# Using the temporary directory
with temporary_directory() as temp_dir:
    # Work with temporary files
    data_file = os.path.join(temp_dir, "experiment_data.txt")
    with open(data_file, 'w') as f:
        f.write("Temperature,Pressure\n")
        f.write("25.0,1013.25\n")
        f.write("26.0,1012.80\n")
    
    # Read it back
    with open(data_file, 'r') as f:
        content = f.read()
        print("File content:")
        print(content)

print("Temporary directory has been cleaned up")

# Directory change context manager
@contextmanager
def change_directory(path: str):
    """Temporarily change directory"""
    old_path = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(old_path)

# Example usage (commented out to avoid changing actual directory)
# with change_directory('/tmp'):
#     print(f"Current directory: {os.getcwd()}")
# print(f"Back to original directory: {os.getcwd()}")

## Generators and Iterators

In [ ]:
# Generator function for sensor readings
def sensor_readings(start_value: float, drift_rate: float, noise_level: float):
    """Generate infinite stream of sensor readings with drift and noise"""
    import random
    current_value = start_value
    reading_count = 0
    
    while True:
        # Add drift and noise
        noise = random.uniform(-noise_level, noise_level)
        current_value += drift_rate + noise
        reading_count += 1
        
        yield {
            'reading': reading_count,
            'value': current_value,
            'timestamp': time.time()
        }

# Use generator
temp_sensor = sensor_readings(25.0, 0.01, 0.1)

print("First 5 temperature readings:")
for i, reading in enumerate(temp_sensor):
    if i >= 5:
        break
    print(f"Reading {reading['reading']}: {reading['value']:.2f}°C")

# Generator expression for data processing
raw_data = [23.1, 24.5, 22.8, 25.2, 26.1, 23.7, 24.3]
filtered_data = (x for x in raw_data if 23.5 <= x <= 25.5)
scaled_data = (x * 1.8 + 32 for x in filtered_data)  # Convert to Fahrenheit

print("\nFiltered and scaled data (F):")
for temp_f in scaled_data:
    print(f"{temp_f:.1f}°F")

# Custom iterator class
class ExperimentBatch:
    """Iterator for processing experimental batches"""
    
    def __init__(self, data_list: List[float], batch_size: int = 3):
        self.data = data_list
        self.batch_size = batch_size
        self.current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= len(self.data):
            raise StopIteration
        
        batch = self.data[self.current:self.current + self.batch_size]
        self.current += self.batch_size
        
        return {
            'batch_data': batch,
            'batch_mean': sum(batch) / len(batch),
            'batch_size': len(batch)
        }

# Use custom iterator
experiment_data = [1.2, 1.5, 1.1, 1.8, 1.3, 1.7, 1.4, 1.6, 1.9, 1.0]
batch_processor = ExperimentBatch(experiment_data, batch_size=3)

print("\nProcessing in batches:")
for batch_info in batch_processor:
    print(f"Batch: {batch_info['batch_data']}, Mean: {batch_info['batch_mean']:.2f}")

# Generator for file processing
def process_large_dataset(filename: str):
    """Generator for processing large files line by line"""
    try:
        with open(filename, 'r') as file:
            for line_number, line in enumerate(file, 1):
                # Process line
                cleaned_line = line.strip()
                if cleaned_line and not cleaned_line.startswith('#'):
                    yield line_number, cleaned_line
    except FileNotFoundError:
        print(f"File {filename} not found")

# Generator pipeline example
def fibonacci_generator(max_value: int):
    """Generate Fibonacci sequence up to max_value"""
    a, b = 0, 1
    while a <= max_value:
        yield a
        a, b = b, a + b

def square_generator(numbers):
    """Square each number from input generator"""
    for num in numbers:
        yield num ** 2

def filter_even(numbers):
    """Filter even numbers"""
    for num in numbers:
        if num % 2 == 0:
            yield num

# Chain generators together
fib_nums = fibonacci_generator(20)
squared_nums = square_generator(fib_nums)
even_squares = filter_even(squared_nums)

print("\nEven squares of Fibonacci numbers ≤ 20:")
print(list(even_squares))

## Advanced Object-Oriented Programming

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Protocol

# Abstract base class
class Instrument(ABC):
    """Abstract base class for laboratory instruments"""
    
    def __init__(self, name: str, serial_number: str):
        self.name = name
        self.serial_number = serial_number
        self.is_calibrated = False
    
    @abstractmethod
    def measure(self, sample) -> float:
        """Take a measurement"""
        pass
    
    @abstractmethod
    def calibrate(self) -> bool:
        """Calibrate the instrument"""
        pass
    
    def status(self) -> str:
        """Get instrument status"""
        cal_status = "Calibrated" if self.is_calibrated else "Not calibrated"
        return f"{self.name} ({self.serial_number}): {cal_status}"

# Concrete implementations
class Spectrometer(Instrument):
    """UV-Vis Spectrometer implementation"""
    
    def __init__(self, name: str, serial_number: str, wavelength_range: tuple):
        super().__init__(name, serial_number)
        self.wavelength_range = wavelength_range
        self._reference_blank = None
    
    def set_reference(self, blank_reading: float):
        """Set reference blank"""
        self._reference_blank = blank_reading
    
    def measure(self, sample: Dict[str, Any]) -> float:
        """Measure absorbance"""
        if not self.is_calibrated:
            raise RuntimeError("Instrument not calibrated")
        
        # Simulate measurement
        import random
        base_absorbance = sample.get('concentration', 0) * 0.8
        noise = random.uniform(-0.01, 0.01)
        return base_absorbance + noise
    
    def calibrate(self) -> bool:
        """Calibrate spectrometer"""
        print(f"Calibrating {self.name}...")
        # Simulate calibration
        self.is_calibrated = True
        return True

class pHMeter(Instrument):
    """pH Meter implementation"""
    
    def __init__(self, name: str, serial_number: str):
        super().__init__(name, serial_number)
        self.buffer_calibrations = {}
    
    def measure(self, sample: Dict[str, Any]) -> float:
        if not self.is_calibrated:
            raise RuntimeError("Instrument not calibrated")
        
        # Simulate pH measurement
        import random
        base_ph = sample.get('ph', 7.0)
        noise = random.uniform(-0.05, 0.05)
        return base_ph + noise
    
    def calibrate(self) -> bool:
        """Two-point calibration with pH 4 and pH 7 buffers"""
        print(f"Calibrating {self.name} with pH buffers...")
        self.buffer_calibrations = {'pH4': 4.01, 'pH7': 7.00}
        self.is_calibrated = True
        return True

# Dataclass for experimental data
@dataclass
class Sample:
    """Represents an experimental sample"""
    id: str
    concentration: float
    ph: float = 7.0
    temperature: float = 25.0
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def __post_init__(self):
        if self.concentration < 0:
            raise ValueError("Concentration cannot be negative")
    
    @property
    def is_acidic(self) -> bool:
        return self.ph < 7.0
    
    @property
    def is_basic(self) -> bool:
        return self.ph > 7.0

# Protocol (structural typing)
class Measurable(Protocol):
    """Protocol for objects that can be measured"""
    def measure(self, sample) -> float: ...

# Laboratory class using composition
class Laboratory:
    """Manages multiple instruments and experiments"""
    
    def __init__(self, name: str):
        self.name = name
        self.instruments: List[Instrument] = []
        self.samples: List[Sample] = []
    
    def add_instrument(self, instrument: Instrument):
        """Add instrument to laboratory"""
        self.instruments.append(instrument)
        print(f"Added {instrument.name} to {self.name}")
    
    def calibrate_all(self):
        """Calibrate all instruments"""
        for instrument in self.instruments:
            instrument.calibrate()
    
    def run_experiment(self, sample: Sample, instrument_type: type) -> Dict[str, Any]:
        """Run experiment with specific instrument type"""
        # Find instrument of specified type
        instrument = None
        for instr in self.instruments:
            if isinstance(instr, instrument_type):
                instrument = instr
                break
        
        if not instrument:
            raise ValueError(f"No {instrument_type.__name__} available")
        
        if not instrument.is_calibrated:
            instrument.calibrate()
        
        measurement = instrument.measure(sample.__dict__)
        
        return {
            'sample_id': sample.id,
            'instrument': instrument.name,
            'measurement': measurement,
            'timestamp': time.time()
        }

# Demonstration
lab = Laboratory("Analytical Chemistry Lab")

# Create instruments
spec = Spectrometer("UV-2600", "SN12345", (190, 1100))
ph_meter = pHMeter("Orion Star", "SN67890")

# Add to lab
lab.add_instrument(spec)
lab.add_instrument(ph_meter)

# Create samples
sample1 = Sample("S001", concentration=0.5, ph=6.8)
sample2 = Sample("S002", concentration=0.3, ph=7.2, metadata={'buffer': 'phosphate'})

print("\nInstrument status:")
for instrument in lab.instruments:
    print(instrument.status())

# Run experiments
print("\nRunning experiments:")
result1 = lab.run_experiment(sample1, Spectrometer)
result2 = lab.run_experiment(sample1, pHMeter)

print(f"Absorbance measurement: {result1['measurement']:.3f}")
print(f"pH measurement: {result2['measurement']:.2f}")

print(f"\nSample S001 is {'acidic' if sample1.is_acidic else 'basic' if sample1.is_basic else 'neutral'}")

## Advanced Data Structures

In [ ]:
from collections import namedtuple, deque, defaultdict, Counter, ChainMap
from enum import Enum, auto
import heapq

# Named tuples for structured data
Measurement = namedtuple('Measurement', ['value', 'unit', 'timestamp', 'uncertainty'])
Coordinate = namedtuple('Coordinate', ['x', 'y', 'z'])

# Create measurements
temp_reading = Measurement(25.4, '°C', time.time(), 0.1)
pressure_reading = Measurement(1013.25, 'hPa', time.time(), 0.5)

print(f"Temperature: {temp_reading.value} ± {temp_reading.uncertainty} {temp_reading.unit}")
print(f"Pressure: {pressure_reading.value} {pressure_reading.unit}")

# Deque for sliding window analysis
class SlidingWindow:
    """Sliding window for real-time data analysis"""
    
    def __init__(self, max_size: int):
        self.data = deque(maxlen=max_size)
        self.max_size = max_size
    
    def add(self, value: float):
        """Add new value to window"""
        self.data.append(value)
    
    @property
    def mean(self) -> float:
        """Calculate mean of current window"""
        return sum(self.data) / len(self.data) if self.data else 0
    
    @property
    def is_full(self) -> bool:
        return len(self.data) == self.max_size

# Test sliding window
window = SlidingWindow(5)
test_data = [23.1, 24.5, 22.8, 25.2, 26.1, 23.7, 24.3]

print("\nSliding window analysis:")
for i, value in enumerate(test_data):
    window.add(value)
    print(f"Step {i+1}: Added {value}, Window: {list(window.data)}, Mean: {window.mean:.2f}")

# Default dictionary for grouping data
experiment_data = defaultdict(list)

# Simulate experimental results
results = [
    ('control', 15.2), ('treatment_A', 23.4), ('control', 14.8),
    ('treatment_B', 18.7), ('treatment_A', 22.1), ('control', 16.1)
]

for group, value in results:
    experiment_data[group].append(value)

print("\nGrouped experimental data:")
for group, values in experiment_data.items():
    mean_val = sum(values) / len(values)
    print(f"{group}: {values} (mean: {mean_val:.1f})")

# Counter for frequency analysis
amino_acids = "ALANYLGLYCYLVALINEISOLEUCINE"
aa_frequency = Counter(amino_acids)

print("\nAmino acid frequency:")
for aa, count in aa_frequency.most_common():
    print(f"{aa}: {count}")

# ChainMap for configuration hierarchy
default_config = {
    'temperature': 25.0,
    'pressure': 1013.25,
    'ph': 7.0,
    'stirring_speed': 500
}

experiment_config = {
    'temperature': 37.0,
    'ph': 6.8
}

user_config = {
    'stirring_speed': 750
}

# Chain configurations (user overrides experiment overrides default)
final_config = ChainMap(user_config, experiment_config, default_config)

print("\nConfiguration hierarchy:")
for key, value in final_config.items():
    print(f"{key}: {value}")

# Enums for experimental conditions
class ExperimentPhase(Enum):
    PREPARATION = auto()
    BASELINE = auto()
    TREATMENT = auto()
    RECOVERY = auto()
    CLEANUP = auto()

class InstrumentStatus(Enum):
    OFFLINE = "offline"
    IDLE = "idle"
    MEASURING = "measuring"
    CALIBRATING = "calibrating"
    ERROR = "error"

# Using enums
current_phase = ExperimentPhase.BASELINE
spectrometer_status = InstrumentStatus.MEASURING

print(f"\nCurrent experiment phase: {current_phase.name}")
print(f"Spectrometer status: {spectrometer_status.value}")

# Priority queue for task scheduling
class Task:
    def __init__(self, priority: int, description: str, estimated_time: float):
        self.priority = priority
        self.description = description
        self.estimated_time = estimated_time
    
    def __lt__(self, other):
        return self.priority < other.priority
    
    def __repr__(self):
        return f"Task(priority={self.priority}, desc='{self.description}')"

# Create task queue
task_queue = []

# Add tasks (lower priority number = higher priority)
heapq.heappush(task_queue, Task(1, "Calibrate instruments", 10.0))
heapq.heappush(task_queue, Task(3, "Clean glassware", 15.0))
heapq.heappush(task_queue, Task(2, "Prepare samples", 20.0))
heapq.heappush(task_queue, Task(1, "Check safety equipment", 5.0))

print("\nTask execution order:")
while task_queue:
    task = heapq.heappop(task_queue)
    print(f"Execute: {task.description} (Priority: {task.priority}, Time: {task.estimated_time} min)")

## Metaprogramming and Introspection

In [ ]:
import inspect
from types import SimpleNamespace

# Dynamic class creation
def create_measurement_class(measurement_type: str, units: str, precision: int = 2):
    """Dynamically create measurement classes"""
    
    def __init__(self, value: float, uncertainty: float = 0.0):
        self.value = round(value, precision)
        self.uncertainty = round(uncertainty, precision)
        self.units = units
    
    def __str__(self):
        if self.uncertainty > 0:
            return f"{self.value} ± {self.uncertainty} {self.units}"
        return f"{self.value} {self.units}"
    
    def __add__(self, other):
        if hasattr(other, 'units') and other.units != self.units:
            raise ValueError(f"Cannot add {self.units} and {other.units}")
        new_value = self.value + (other.value if hasattr(other, 'value') else other)
        new_uncertainty = (self.uncertainty**2 + getattr(other, 'uncertainty', 0)**2)**0.5
        return cls(new_value, new_uncertainty)
    
    # Create class attributes
    class_dict = {
        '__init__': __init__,
        '__str__': __str__,
        '__add__': __add__,
        'measurement_type': measurement_type,
        'default_units': units
    }
    
    # Create the class
    cls = type(f"{measurement_type}Measurement", (object,), class_dict)
    return cls

# Create measurement classes dynamically
Temperature = create_measurement_class("Temperature", "°C", 1)
Concentration = create_measurement_class("Concentration", "mol/L", 4)
Time = create_measurement_class("Time", "s", 2)

# Use dynamically created classes
temp1 = Temperature(25.3, 0.1)
temp2 = Temperature(26.1, 0.2)
conc = Concentration(0.0015, 0.0001)

print(f"Temperature 1: {temp1}")
print(f"Temperature 2: {temp2}")
print(f"Sum: {temp1 + temp2}")
print(f"Concentration: {conc}")

# Introspection examples
def analyze_function(func):
    """Analyze function signature and metadata"""
    sig = inspect.signature(func)
    
    print(f"\nFunction: {func.__name__}")
    print(f"Docstring: {func.__doc__}")
    print(f"Module: {func.__module__}")
    print(f"Parameters:")
    
    for name, param in sig.parameters.items():
        print(f"  {name}: {param.annotation if param.annotation != param.empty else 'no annotation'}")
        if param.default != param.empty:
            print(f"    Default: {param.default}")
    
    if sig.return_annotation != sig.empty:
        print(f"Return type: {sig.return_annotation}")

# Analyze our temperature function
analyze_function(calculate_concentration)

# Property decorator with validation
def validated_property(validator_func):
    """Decorator factory for creating validated properties"""
    def decorator(func):
        name = f"_{func.__name__}"
        
        def getter(self):
            return getattr(self, name, None)
        
        def setter(self, value):
            if not validator_func(value):
                raise ValueError(f"Invalid value for {func.__name__}: {value}")
            setattr(self, name, value)
        
        return property(getter, setter)
    return decorator

# Custom class with validated properties
class ExperimentalSetup:
    """Experimental setup with validated parameters"""
    
    @validated_property(lambda x: 0 <= x <= 100)
    def temperature(self):
        """Temperature in Celsius (0-100)"""
        pass
    
    @validated_property(lambda x: 0 < x <= 14)
    def ph(self):
        """pH value (0-14)"""
        pass
    
    @validated_property(lambda x: x >= 0)
    def concentration(self):
        """Concentration (non-negative)"""
        pass

# Test validated properties
setup = ExperimentalSetup()
setup.temperature = 25.0
setup.ph = 7.2
setup.concentration = 0.5

print(f"\nExperimental setup:")
print(f"Temperature: {setup.temperature}°C")
print(f"pH: {setup.ph}")
print(f"Concentration: {setup.concentration} mol/L")

# Try invalid value
try:
    setup.temperature = 150  # Invalid (> 100)
except ValueError as e:
    print(f"Validation error: {e}")

# Metaclass example
class InstrumentMeta(type):
    """Metaclass that adds automatic registration of instruments"""
    
    _registry = {}
    
    def __new__(cls, name, bases, namespace):
        # Add automatic ID generation
        if 'instrument_id' not in namespace:
            namespace['instrument_id'] = f"{name.lower()}_{len(cls._registry) + 1}"
        
        # Create the class
        new_class = super().__new__(cls, name, bases, namespace)
        
        # Register the class
        if name != 'BaseInstrument':  # Don't register base class
            cls._registry[name] = new_class
        
        return new_class
    
    @classmethod
    def get_registered_instruments(cls):
        return cls._registry

class BaseInstrument(metaclass=InstrumentMeta):
    """Base instrument class with automatic registration"""
    pass

class Microscope(BaseInstrument):
    pass

class Centrifuge(BaseInstrument):
    pass

# Check registration
print("\nRegistered instruments:")
for name, cls in InstrumentMeta.get_registered_instruments().items():
    instrument = cls()
    print(f"{name}: ID = {instrument.instrument_id}")

# Dynamic attribute access
config = SimpleNamespace()
config.temperature = 25.0
config.pressure = 1013.25
config.humidity = 45.0

print("\nDynamic configuration:")
for attr_name in dir(config):
    if not attr_name.startswith('_'):
        value = getattr(config, attr_name)
        print(f"{attr_name}: {value}")

# Add new attribute dynamically
setattr(config, 'ph', 7.0)
print(f"Added pH: {config.ph}")

## Concurrency and Performance

In [ ]:
import threading
import queue
import concurrent.futures
import multiprocessing
from functools import lru_cache
import weakref

# Thread-safe data collection
class ThreadSafeDataCollector:
    """Thread-safe data collector for concurrent measurements"""
    
    def __init__(self):
        self._data = []
        self._lock = threading.Lock()
        self._measurements_count = 0
    
    def add_measurement(self, value: float, source: str):
        """Thread-safe method to add measurements"""
        with self._lock:
            timestamp = time.time()
            self._data.append({
                'value': value,
                'source': source,
                'timestamp': timestamp,
                'measurement_id': self._measurements_count
            })
            self._measurements_count += 1
    
    def get_data(self):
        """Get copy of all data"""
        with self._lock:
            return self._data.copy()
    
    def get_stats(self):
        """Get basic statistics"""
        with self._lock:
            if not self._data:
                return {}
            
            values = [d['value'] for d in self._data]
            return {
                'count': len(values),
                'mean': sum(values) / len(values),
                'min': min(values),
                'max': max(values)
            }

# Simulate sensor reading function
def simulate_sensor_reading(sensor_id: str, collector: ThreadSafeDataCollector, num_readings: int = 5):
    """Simulate sensor taking multiple readings"""
    import random
    
    for i in range(num_readings):
        # Simulate measurement time
        time.sleep(random.uniform(0.01, 0.05))
        
        # Generate reading with some variation
        base_value = {'temp': 25.0, 'pressure': 1013.0, 'ph': 7.0}.get(sensor_id, 1.0)
        reading = base_value + random.uniform(-1, 1)
        
        collector.add_measurement(reading, sensor_id)
        print(f"{sensor_id}: Reading {i+1} = {reading:.2f}")

# Concurrent data collection
print("Concurrent sensor readings:")
collector = ThreadSafeDataCollector()

# Create and start threads
threads = []
sensors = ['temp', 'pressure', 'ph']

for sensor in sensors:
    thread = threading.Thread(target=simulate_sensor_reading, args=(sensor, collector, 3))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

print("\nCollection complete. Statistics:")
stats = collector.get_stats()
for key, value in stats.items():
    print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")

# Producer-Consumer pattern with Queue
def data_producer(data_queue: queue.Queue, num_items: int = 10):
    """Produce data and put in queue"""
    import random
    
    for i in range(num_items):
        data = {
            'sample_id': f'S{i+1:03d}',
            'value': random.uniform(20, 30),
            'priority': random.randint(1, 5)
        }
        data_queue.put(data)
        time.sleep(0.01)  # Simulate production time
    
    # Signal completion
    data_queue.put(None)

def data_consumer(data_queue: queue.Queue, consumer_id: str):
    """Consume data from queue"""
    processed = 0
    
    while True:
        try:
            data = data_queue.get(timeout=1)
            if data is None:
                break
            
            # Simulate processing
            time.sleep(0.02)
            processed += 1
            print(f"Consumer {consumer_id}: Processed {data['sample_id']} (value: {data['value']:.1f})")
            
            data_queue.task_done()
            
        except queue.Empty:
            break
    
    print(f"Consumer {consumer_id} finished. Processed {processed} items.")

# Demonstrate producer-consumer
print("\nProducer-Consumer pattern:")
data_queue = queue.Queue(maxsize=5)

# Start producer and consumers
producer_thread = threading.Thread(target=data_producer, args=(data_queue,))
consumer1_thread = threading.Thread(target=data_consumer, args=(data_queue, "A"))
consumer2_thread = threading.Thread(target=data_consumer, args=(data_queue, "B"))

producer_thread.start()
consumer1_thread.start()
consumer2_thread.start()

producer_thread.join()
consumer1_thread.join()
consumer2_thread.join()

# CPU-intensive task for multiprocessing
def analyze_spectrum_data(data_chunk):
    """CPU-intensive spectral analysis simulation"""
    import math
    
    result = 0
    for value in data_chunk:
        # Simulate complex calculation
        result += math.sin(value) * math.cos(value * 2) + math.sqrt(abs(value))
    
    return {
        'chunk_size': len(data_chunk),
        'analysis_result': result,
        'mean_value': sum(data_chunk) / len(data_chunk)
    }

# Parallel processing with ThreadPoolExecutor
print("\nParallel processing example:")
import random

# Generate large dataset
large_dataset = [random.uniform(0, 100) for _ in range(1000)]

# Split into chunks
chunk_size = 100
chunks = [large_dataset[i:i+chunk_size] for i in range(0, len(large_dataset), chunk_size)]

# Process in parallel
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    future_to_chunk = {executor.submit(analyze_spectrum_data, chunk): i 
                      for i, chunk in enumerate(chunks)}
    
    results = []
    for future in concurrent.futures.as_completed(future_to_chunk):
        chunk_id = future_to_chunk[future]
        try:
            result = future.result()
            results.append(result)
            print(f"Chunk {chunk_id}: Analysis result = {result['analysis_result']:.2f}")
        except Exception as exc:
            print(f"Chunk {chunk_id} generated an exception: {exc}")

end_time = time.time()
print(f"Parallel processing completed in {end_time - start_time:.2f} seconds")

# Caching for expensive computations
@lru_cache(maxsize=128)
def expensive_calculation(n: int) -> float:
    """Simulate expensive calculation with caching"""
    time.sleep(0.01)  # Simulate computation time
    return sum(i**2 for i in range(n))

# Test caching performance
print("\nCaching performance test:")

# First run (cache miss)
start = time.time()
result1 = expensive_calculation(100)
first_time = time.time() - start

# Second run (cache hit)
start = time.time()
result2 = expensive_calculation(100)
second_time = time.time() - start

print(f"First call: {first_time:.4f}s (result: {result1})")
print(f"Second call: {second_time:.4f}s (result: {result2})")
print(f"Speedup: {first_time/second_time:.1f}x")
print(f"Cache info: {expensive_calculation.cache_info()}")

# Memory management with weak references
class InstrumentManager:
    """Manages instruments with weak references to avoid circular references"""
    
    def __init__(self):
        self._instruments = weakref.WeakValueDictionary()
    
    def register_instrument(self, name: str, instrument):
        """Register instrument with weak reference"""
        self._instruments[name] = instrument
    
    def get_instrument(self, name: str):
        """Get instrument by name"""
        return self._instruments.get(name)
    
    def list_instruments(self):
        """List all registered instruments"""
        return list(self._instruments.keys())

# Demo weak references
class DemoInstrument:
    def __init__(self, name):
        self.name = name
    
    def __del__(self):
        print(f"Instrument {self.name} is being garbage collected")

manager = InstrumentManager()

# Create and register instrument
instrument = DemoInstrument("Spectrometer")
manager.register_instrument("spec1", instrument)

print(f"\nRegistered instruments: {manager.list_instruments()}")

# Delete reference - instrument should be garbage collected
del instrument
import gc
gc.collect()  # Force garbage collection

print(f"Instruments after deletion: {manager.list_instruments()}")